In [1]:
import os
import sys
import numpy as np
import cv2

from data_loader import *
from fbs_config import TrainFBSConfig, InferenceFBSConfig
from fbs_dataset import FBSDataset

from mrcnn import model as modellib
from datahandler import DataHandler

from sklearn.metrics import f1_score
from scipy.ndimage import _ni_support
from scipy.ndimage.morphology import distance_transform_edt, binary_erosion,\
     generate_binary_structure
from tqdm import tqdm

from medpy.io import save
from math import ceil, floor

import skimage.color

from skimage.morphology import cube, binary_closing
from skimage.measure import label

ROOT_DIR = os.path.abspath('../../../')
sys.path.append(ROOT_DIR)

DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, 'logs')
DEFAULT_MODEL_DIR = os.path.join(DEFAULT_LOGS_DIR, 'mask_rcnn/kfold')

kernel = np.ones((5,5),np.uint8)

dh = DataHandler()

Using TensorFlow backend.


In [2]:
def destiny_directory(dice_score, post_processing = False):
    if post_processing:
        pre = './data/eval_pp/mask_rcnn/'
    else:
        pre = './data/eval/mask_rcnn/'

    if dice_score >= 98:
        return pre + 'dice_98_100/'
    elif dice_score >= 96:
        return pre + 'dice_96_98/'
    elif dice_score >= 94:
        return pre + 'dice_94_96/'
    elif dice_score >= 92:
        return pre + 'dice_92_94/'
    elif dice_score >= 90:
        return pre + 'dice_90_92/'
    elif dice_score >= 88:
        return pre + 'dice_88_90/'
    elif dice_score >= 85:
        return pre + 'dice_85_88'
    elif dice_score >= 80:
        return pre + 'dice_80_85/'
    elif dice_score >= 70:
        return pre + 'dice_70_80/'
    elif dice_score >= 60:
        return pre + 'dice_60_70/'
    else:
        return pre + 'dice_less_60'

def getFileName(fname):
    original_name = fname.split('/')[-1]
    original_name = original_name[:original_name.index('.')]
    return original_name

In [3]:
image_files, mask_files = load_data_files('data/kfold_data/')

skf = getKFolds(image_files, mask_files, n=10)

kfold_indices = []
for train_index, test_index in skf.split(image_files, mask_files):
    kfold_indices.append({'train': train_index, 'val': test_index})

In [4]:
def getDataset(val_index):
    image_val_files = np.take(image_files, val_index)
    mask_val_files = np.take(mask_files, val_index)

    val_files = ([image_val_files], [mask_val_files])

    dataset_val = FBSDataset()
    len_dataset_val = dataset_val.load_data(val_files)
    dataset_val.prepare()
    return dataset_val

In [5]:
def getDiceScore(ground_truth, prediction):
    #convert to boolean values and flatten
    ground_truth = np.asarray(ground_truth, dtype=np.bool).flatten()
    prediction = np.asarray(prediction, dtype=np.bool).flatten()    
    return f1_score(ground_truth, prediction)

In [6]:
 def hd(result, reference, voxelspacing=None, connectivity=1):
    hd1 = __surface_distances(result, reference, voxelspacing, connectivity).max()
    hd2 = __surface_distances(reference, result, voxelspacing, connectivity).max()
    hd = max(hd1, hd2)
    return hd

def hd95(result, reference, voxelspacing=None, connectivity=1):
    hd1 = __surface_distances(result, reference, voxelspacing, connectivity)
    hd2 = __surface_distances(reference, result, voxelspacing, connectivity)
    hd95 = np.percentile(np.hstack((hd1, hd2)), 95)
    return hd95

def __surface_distances(result, reference, voxelspacing=None, connectivity=1):
    result = np.atleast_1d(result.astype(np.bool))
    reference = np.atleast_1d(reference.astype(np.bool))
    if voxelspacing is not None:
        voxelspacing = _ni_support._normalize_sequence(voxelspacing, result.ndim)
        voxelspacing = np.asarray(voxelspacing, dtype=np.float64)
        if not voxelspacing.flags.contiguous:
            voxelspacing = voxelspacing.copy()

    footprint = generate_binary_structure(result.ndim, connectivity)

    if 0 == np.count_nonzero(result):
        raise RuntimeError('The first supplied array does not contain any binary object.')
    if 0 == np.count_nonzero(reference):
        raise RuntimeError('The second supplied array does not contain any binary object.')

    result_border = result ^ binary_erosion(result, structure=footprint, iterations=1)
    reference_border = reference ^ binary_erosion(reference, structure=footprint, iterations=1)

    dt = distance_transform_edt(~reference_border, sampling=voxelspacing)
    sds = dt[result_border]

    return sds

In [7]:
def evaluateMask(gt_mask, pred_mask):
    return getDiceScore(gt_mask, pred_mask), hd(gt_mask, pred_mask), hd95(gt_mask, pred_mask)
    

In [8]:
import random
def prepareForSaving(image):
    #image = np.swapaxes(image, -1, 0)
    image = np.moveaxis(image, 0, -1)
    
    return image

def predictAll(inferenceFBSConfig, val_indices, post_processing = False):
    model = modellib.MaskRCNN(mode='inference', config=inferenceFBSConfig, model_dir=DEFAULT_MODEL_DIR)
    inferenceFBSConfig.display()
    print(DEFAULT_MODEL_DIR)
    weights_path = model.find_last()
    print('Loading weights from %s'%weights_path)
    model.load_weights(weights_path, by_name=True)
    
    dice_scores = []
    hd_scores = []
    hd95_scores = []
    names = []
    
    for image_index in tqdm(val_indices):
        #for saving
        fname = getFileName(image_files[image_index])
        not_used_full_image, hdr = dh.getImageData(image_files[image_index])
        dataset = getDataset(image_index)
                
        prediction = []
        gt_mask = []
    
        for img_id in dataset.image_ids:
            image, image_meta, class_ids, bbox, mask = modellib.load_image_gt(
            dataset, inferenceFBSConfig, img_id, use_mini_mask=False)
            
            results = model.detect([image], verbose=0)
            r = results[0]
            
            pred = r['masks']
            
            if(len(pred.shape) > 2 and pred.shape[2] == 0):
                pred = np.zeros((pred.shape[0],pred.shape[1],1))
            
            if(mask.shape[2] == 0):
                mask = np.zeros((pred.shape[0],pred.shape[1],1))
            
            pred[pred>=0.5] = 1
            pred[pred<0.5] = 0
        
            pred = np.asarray(pred, dtype=np.uint8)
            pred = cv2.dilate(pred,kernel,iterations = 1)
                     
            prediction.append(pred)
            gt_mask.append(mask)
        
        pred_mask = np.asarray(prediction)
        gt_mask = np.asarray(gt_mask)
        
        gt_mask = np.squeeze(gt_mask)
        pred_mask = np.squeeze(pred_mask)
        
        if post_processing:
            pred_mask = binary_closing(pred_mask, cube(2))

            try:
                labels = label(pred_mask)
                pred_mask = (labels == np.argmax(np.bincount(labels.flat)[1:])+1).astype(int)
            except:
                pred_mask = pred_mask
            
            pred_mask = np.array(pred_mask, dtype=np.uint16)
        
        dice_score, hd_score, hd95_score = evaluateMask(np.squeeze(gt_mask), pred_mask)
        
        
        if dice_score == 0:
            dice_scores.append(dice_score)
            hd_scores.append(200)
            hd95_scores.append(200) 
            names.append(fname)
            
            pred_mask = prepareForSaving(pred_mask)
            save_path = destiny_directory(int_dice_score, post_processing=post_processing)
            save_path = os.path.join(ROOT_DIR, save_path)
            
            save(pred, os.path.join(save_path, fname + '_mask_rcnn_' 
                + str(int_dice_score) + '.nii'), hdr)
            continue
        
        names.append(fname)
        dice_scores.append(dice_score)
        hd_scores.append(hd_score)
        hd95_scores.append(hd95_score)
        
        int_dice_score = floor(dice_score * 100)
        
        pred_mask = prepareForSaving(pred_mask)
        save_path = destiny_directory(int_dice_score, post_processing=post_processing)
        save_path = os.path.join(ROOT_DIR, save_path) 
        save(pred_mask, os.path.join(save_path, fname + '_mask_rcnn_' 
            + str(int_dice_score) + '.nii'), hdr)
    
    return dice_scores, hd_scores, hd95_scores, names

In [9]:
all_dice = []
all_hd = []
all_hd95 = []
all_names = []

for post_processing in [False, True]:

    for i in range(10):#len(kfold_indices)):    

        configParams = {'da': True,'tl': True, 'mask_dim': 28, 'wl': True, 'kfold_i': i}

        trainFBSConfig = TrainFBSConfig(**configParams)
        inferenceFBSConfig = InferenceFBSConfig(**configParams)    
        print(inferenceFBSConfig.display())
        dice_scores, hd_scores, hd95_scores, names = predictAll(inferenceFBSConfig, 
                                                         kfold_indices[i]['val'],
                                                        post_processing = post_processing)

        print('Finished K%d'%i)
        
        all_dice += dice_scores
        all_hd += hd_scores
        all_hd95 += hd95_scores
        all_names.extend(names)


    if post_processing:
        report_name = 'data/eval_pp/mask_rcnn/mask_rcnn_report.txt'

    else:
        report_name = 'data/eval/mask_rcnn/mask_rcnn_report.txt'
        
    report_name = os.path.join(ROOT_DIR, report_name)
    
    with open(report_name, 'w+') as f:
        for i in range(len(all_dice)):
            f.write("%s, %f, %f, %f\n"%(all_names[i],
                                      all_dice[i],
                                      all_hd[i],
                                      all_hd95[i]))

        f.write('\n')
        f.write('Final results for mask_rcnn\n')
        f.write('dice %f\n'%np.mean(all_dice))
        f.write('hd %f\n'%np.mean(all_hd))
        f.write('hd95 %f\n'%np.mean(all_hd95))


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        1
DETECTION_MIN_CONFIDENCE       0
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            1
IMAGE_MAX_DIM                  256
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  256
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [256 256   1]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 0.5, 'rpn_bbox_loss': 0.5, 'mrcnn_class_loss': 0.5, 'mrcnn_bbox_loss': 0.5, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                  

  0%|          | 0/30 [00:00<?, ?it/s]

Re-starting from epoch 23



100%|██████████| 30/30 [04:48<00:00,  8.80s/it]53s/it]


Finished K0

Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        1
DETECTION_MIN_CONFIDENCE       0
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            1
IMAGE_MAX_DIM                  256
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  256
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [256 256   1]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 0.5, 'rpn_bbox_loss': 0.5, 'mrcnn_class_loss': 0.5, 'mrcnn_bbox_loss': 0.5, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE      

...

Finished K8

Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        1
DETECTION_MIN_CONFIDENCE       0
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            1
IMAGE_MAX_DIM                  256
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  256
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [256 256   1]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 0.5, 'rpn_bbox_loss': 0.5, 'mrcnn_class_loss': 0.5, 'mrcnn_bbox_loss': 0.5, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE      

  0%|          | 0/29 [00:00<?, ?it/s]

Re-starting from epoch 13



100%|██████████| 29/29 [05:03<00:00,  9.18s/it]75s/it]

Finished K9


In [10]:
print('dice')
for score in all_dice:
    print(score)

print()
print('hd')
for score in all_hd:
    print(score)

print()
print('hd95')
for score in all_hd95:
    print(score)
    

dice
0.8667874675370065
0.9023389743259765
0.7726390412693054
0.8380440336290585
0.7104639260359914
0.9327499728422717
0.9336195695158466
0.8736065362604198
0.9111669938819459
0.8891609323631913
0.9176977470037441
0.8985425390619829
0.8762968531844055
0.9267180202583853
0.9180760020049381
0.903923979304339
0.9057285160629064
0.6320465273095077
0.8800461730849392
0.902440304382052
0.22591043939358543
0.9318017405791701
0.9393663145222497
0.92815591711231
0.9242727070841661
0.8801207196749855
0.9163887212995322
0.8898514642348183
0.8789906985590464
0.8631725738374166
0.8926971108381133
0.9093438441919777
0.917081131488965
0.9172673614170005
0.9124013475023919
0.9353343896678525
0.7687518205651034
0.9293593349979455
0.5854852459918048
0.9069019398326204
0.8985705113539921
0.911144340664708
0.9077334453678539
0.7915429771393118
0.901391440002013
0.8734693877551021
0.9468165350835959
0.9517858709960509
0.9227259873692147
0.9322151760001814
0.9419780202509165
0.944007629778315
0.937244436120